In [2]:
import sys
sys.path.insert(0, "/cndd/fangming/CEMBA/snmcseq_dev")

from multiprocessing import Pool,cpu_count
from functools import partial

from scipy import sparse
import importlib
from __init__ import *
from __init__jupyterlab import *
import snmcseq_utils
importlib.reload(snmcseq_utils)

import pickle
import datetime

- read enhancer-cell-matrix (mcg)
- read gene-cell matrix (rna)
- read kNN matrix (from previous files)
- read enhancer-gene linkage files (from previous files)

- do the analysis

# Todo

- figure out the best matrix multiplications to do

- make sure the enhancer-gene linkage file match

In [3]:
today = datetime.date.today()
print(today)

2020-07-28


In [4]:
# check memory usage
# how can it be speed up?
# each enhancer matrix costs ~ 10GB = E5 (regions) E4 (cells) E1 (float64 type costs 8 bytes)
# example_sizes = {
#     "int": 3,
#     "float": 3.0, 
#     "str": "3", 
#     "list": [],
#     "list2": [3,],
#     "tuple": (),
#     "tuple2": (3,),
#     "np_float32": np.float32(3.0),
#     "np_float64": np.float64(3.0),
# }

# for key, val in example_sizes.items():
#     print(key, sys.getsizeof(val))
    
# print(sys.getsizeof(A), A.size, A.itemsize)

# Load data (pull from everywhere)

### old-to-new cell ids (mC) 

In [5]:
# old - new names
f = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze/snmc_and_snatac/metadata/MOp_Metadata.tsv.gz'
old2new = pd.read_csv(f, sep="\t")
old2new['old_name'] = old2new['AllcPath'].apply(lambda x: x.split('/')[-1][len("allc_"):-len(".tsv.gz")])
old2new['new_name'] = old2new['cell'].apply(lambda x: 'snmcseq_gene_{}'.format(x))

print(old2new.shape)
old2new.head()

(9941, 24)


cell                                           AllcPath  CCC_Rate  \
0     2C_M_0  /gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...   0.00511   
1     2C_M_1  /gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...   0.00531   
2    2C_M_10  /gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...   0.00581   
3   2C_M_100  /gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...   0.00634   
4  2C_M_1000  /gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...   0.00521   

   CG_Rate  CG_RateAdj  CH_Rate  CH_RateAdj  FinalReads  InputReads  \
0  0.77073     0.76955  0.02154     0.01651   1076259.0     2314862   
1  0.75683     0.75553  0.02196     0.01674   1197092.0     2590626   
2  0.75720     0.75578  0.02603     0.02034   1006630.0     2348484   
3  0.74984     0.74824  0.02718     0.02097   2026078.0     4599922   
4  0.77073     0.76953  0.02117     0.01604   2368464.0     5320502   

   MappedReads  ... MappingRate Pos96             Plate  Col96  Row96 Col384  \
0    1517338.0  ...    0.655477   A10  CEMBA180409_2C_1      9      0     18   
1    1709334.0  ...    0.659815   A10  CEMBA180409_2C_1      9      0     18   
2    1606410.0  ...    0.684020   A11  CEMBA180409_2C_2     10      0     20   
3    3154619.0  ...    0.685798    B1  CEMBA180409_2C_2      0      1      1   
4    3648786.0  ...    0.685797    F8  CEMBA180409_2C_3      7      5     15   

  Row384           Batch                                           old_name  \
0      0  CEMBA180409_2C  180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...   
1      1  CEMBA180409_2C  180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...   
2      0  CEMBA180409_2C  180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...   
3      2  CEMBA180409_2C  180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...   
4     11  CEMBA180409_2C  180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_3_CE...   

                 new_name  
0     snmcseq_gene_2C_M_0  
1     snmcseq_gene_2C_M_1  
2    snmcseq_gene_2C_M_10  
3   snmcseq_gene_2C_M_100  
4  snmcseq_gene_2C_M_1000  

[5 rows x 24 columns]

### cell-cell knn matrix

In [6]:
# knn mat - not saved - need to regenerate
ti = time.time()
f = '/cndd2/fangming/projects/miniatlas/results/knn_across_mop_2mods_200727_test_smarter_cells_snmcseq_gene.npz'
# 20 (mC cells) for each row (rna cell)
cell_cell_knn_mat = sparse.load_npz(f)
print(cell_cell_knn_mat.shape)
print(time.time()-ti)

cell_cell_knn_mat.sum(axis=1)

f = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/smarter_cells_features.cell'
cell_cell_knn_xaxis = snmcseq_utils.import_single_textcol(f)
print(cell_cell_knn_xaxis.shape, cell_cell_knn_xaxis[:5])

f = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_features.cell'
cell_cell_knn_yaxis = snmcseq_utils.import_single_textcol(f)
print(cell_cell_knn_yaxis.shape, cell_cell_knn_yaxis[:5])


# resolve dimension mismatch

(6244, 9366)
0.08101582527160645
(6244,) ['smarter_cells_LS-15395_S41_E1-50' 'smarter_cells_LS-15395_S42_E1-50'
 'smarter_cells_LS-15501_S28_E1-50' 'smarter_cells_LS-15501_S81_E1-50'
 'smarter_cells_LS-15524_S21_E1-50']
(9366,) ['snmcseq_gene_2C_M_0' 'snmcseq_gene_2C_M_1' 'snmcseq_gene_2C_M_100'
 'snmcseq_gene_2C_M_1000' 'snmcseq_gene_2C_M_1001']


In [7]:
# knn mat - not saved - need to regenerate
ti = time.time()
f = '/cndd2/fangming/projects/miniatlas/results/knn_within_mop_2mods_200727_test_smarter_cells.npz'
# 20 (mC cells) for each row (rna cell)
cell_cell_knn_mat_within = sparse.load_npz(f)
print(cell_cell_knn_mat_within.shape)
print(time.time()-ti)

cell_cell_knn_mat_within.sum(axis=1)

f = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/smarter_cells_features.cell'
cell_cell_knn_mat_within_axis = snmcseq_utils.import_single_textcol(f)
print(cell_cell_knn_mat_within_axis.shape, cell_cell_knn_mat_within_axis[:5])

# resolve dimension mismat_withinch

(6244, 6244)
0.15384650230407715
(6244,) ['smarter_cells_LS-15395_S41_E1-50' 'smarter_cells_LS-15395_S42_E1-50'
 'smarter_cells_LS-15501_S28_E1-50' 'smarter_cells_LS-15501_S81_E1-50'
 'smarter_cells_LS-15524_S21_E1-50']


### enhancer-cell matrix (mC)

In [8]:
ti = time.time()

# takes ~3min to load
# enhancer-cell-matrix
f = '/cndd2/fangming/projects/scf_enhancers/enhancer_cell_level/mcg/combined_enh_cell_mc_mats.pkl'
with open(f, 'rb') as handle:
    enhancer_cell_mc_mats = pickle.load(handle)
    
for col in enhancer_cell_mc_mats.keys():
    # was originally cell-by-enhancer; turn into enhancer-by-cell
    enhancer_cell_mc_mats[col] = enhancer_cell_mc_mats[col].T 
    print(col, enhancer_cell_mc_mats[col].shape)
print(time.time()-ti)

mCG (233514, 9931)
CG (233514, 9931)
mCH (233514, 9931)
CH (233514, 9931)
127.2348256111145


In [9]:
f = '/cndd2/fangming/projects/scf_enhancers/enhancer_cell_level/mcg/combined_enh_cell_mc_mats.cell'
enhancer_cell_mc_mats_cell = snmcseq_utils.import_single_textcol(f)
print(len(enhancer_cell_mc_mats_cell))
enhancer_cell_mc_mats_cell = old2new.set_index('old_name').reindex(enhancer_cell_mc_mats_cell)['new_name'].values
print(len(enhancer_cell_mc_mats_cell))
enhancer_cell_mc_mats_cell

9931
9931


array(['snmcseq_gene_3C_M_0', 'snmcseq_gene_3C_M_1', 'snmcseq_gene_3C_M_2',
       ..., 'snmcseq_gene_5D_M_2370', 'snmcseq_gene_5D_M_2371',
       'snmcseq_gene_5D_M_2372'], dtype=object)

In [10]:
f = '/cndd2/fangming/projects/scf_enhancers/enhancer_cell_level/mcg/combined_enh_cell_mc_mats.region'
enhancer_cell_mc_mats_region = pd.read_csv(f, sep='\t') 
print(len(enhancer_cell_mc_mats_region))
enhancer_cell_mc_mats_region.head()

233514


chr    start      end
0   1  3083769  3083816
1   1  3112234  3112467
2   1  3112770  3112986
3   1  3113717  3114035
4   1  3118790  3118935

### enhancer-cell matrix (RNA)

In [11]:
# gene-cell matrix (rna)

ti = time.time()
f = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/smarter_cells_features.npz'
fgene = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/smarter_cells_features.gene'
fcell = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/smarter_cells_features.cell'

gene_cell_rna_mat = snmcseq_utils.load_gc_matrix(fgene, fcell, f)

print(gene_cell_rna_mat.data.shape)
print(time.time()-ti)


(32324, 6244)
11.233612537384033


### enhancer-gene linkage

In [12]:
# enhancer-gene linkage
ti = time.time()
f = '/cndd2/fangming/projects/scf_enhancers/results/200521_to_evals.tsv'
enhancer_gene_to_eval = pd.read_csv(f, sep='\t')
f = '/cndd2/fangming/projects/scf_enhancers/results/200521_to_evals_shuffled.tsv'
enhancer_gene_to_eval_shuffled = pd.read_csv(f, sep='\t')

print(enhancer_gene_to_eval.shape)
print(time.time()-ti)
enhancer_gene_to_eval.head()

(2589994, 18)
14.266006708145142


gene     ens       val      val2  val_clustershuffled  \
0  ENSMUSG00000000001  147520 -0.157895  0.127913            -0.169712   
1  ENSMUSG00000000001  147522  0.027465 -0.307145             0.064887   
2  ENSMUSG00000000001  147523 -0.040376  0.011489            -0.155487   
3  ENSMUSG00000000001  147527 -0.070795  0.111719            -0.088084   
4  ENSMUSG00000000001  147528  0.012583  0.150673            -0.036218   

   val2_clustershuffled    dist   pvalues  fdr<100kb  fdr<200kb  fdr<500kb  \
0             -0.049349  995118  0.288079   0.674683   0.736811   0.827139   
1              0.105591  985740  0.561881   0.830849   0.867082   0.917381   
2              0.014772  983725  0.457686   0.780196   0.825259   0.889141   
3              0.184594  970444  0.411564   0.755088   0.804341   0.874375   
4              0.147609  969372  0.539123   0.819981   0.858026   0.911467   

   fdr<100kb shuffled (cluster label)  pvalues2  fdr2<100kb  fdr2<200kb  \
0                            1.647931  0.383141    0.745977    0.797076   
1                            0.958300  0.935264    0.973809    0.979844   
2                            1.089312  0.580703    0.841673    0.874844   
3                            1.183332  0.410358    0.761056    0.809464   
4                            0.979843  0.345993    0.725803    0.780447   

   fdr2<500kb  fdr2<100kb shuffled (cluster label)  val_both  
0    0.872822                             1.955898 -0.285808  
1    0.987476                             0.959470  0.334610  
2    0.922832                             1.273339 -0.051866  
3    0.880982                             1.812516 -0.182514  
4    0.862033                             2.191506 -0.138090

# Organize data (and save to local folder) 

### match cells 

In [13]:
common_mc_cells = np.intersect1d(enhancer_cell_mc_mats_cell, 
                                 cell_cell_knn_yaxis,
                                ) 
common_rna_cells = np.intersect1d(gene_cell_rna_mat.cell, 
                                 cell_cell_knn_xaxis) 

print(len(common_mc_cells), len(common_rna_cells))

9364 6244


### match gene and enhancers 

In [14]:
# just need first number greater than the second
print(len(gene_cell_rna_mat.gene), len(enhancer_gene_to_eval['gene'].unique()))
print(len(enhancer_cell_mc_mats_region), len(enhancer_gene_to_eval['ens'].unique()))

32324 18988
233514 151577


### match dimensions of matrices 

In [15]:
cell_idx_rna = snmcseq_utils.get_index_from_array(gene_cell_rna_mat.cell, common_rna_cells)
X = gene_cell_rna_mat.data.tocsc()[:,cell_idx_rna]
print(X.shape)

(32324, 6244)


In [16]:
cell_idx_mc = snmcseq_utils.get_index_from_array(enhancer_cell_mc_mats_cell, common_mc_cells)
Y_mcg = enhancer_cell_mc_mats['mCG'].tocsc()[:,cell_idx_mc]
Y_cg = enhancer_cell_mc_mats['CG'].tocsc()[:,cell_idx_mc]
print(Y_mcg.shape, Y_cg.shape)

(233514, 9364) (233514, 9364)


In [17]:
cell_idx_xaxis = snmcseq_utils.get_index_from_array(cell_cell_knn_xaxis, common_rna_cells)
cell_idx_yaxis = snmcseq_utils.get_index_from_array(cell_cell_knn_yaxis, common_mc_cells)

knn_xy = cell_cell_knn_mat.tocsr()[cell_idx_xaxis,:].tocsc()[:,cell_idx_yaxis] # x-by-y
print(knn_xy.shape)

(6244, 9364)


In [18]:
cell_idx_axis = snmcseq_utils.get_index_from_array(cell_cell_knn_mat_within_axis, common_rna_cells)
knn_xx = cell_cell_knn_mat_within.tocsr()[cell_idx_axis,:].tocsc()[:,cell_idx_axis] # x-by-x
print(knn_xx.shape)

(6244, 6244)


In [21]:
print(X.shape, Y_mcg.shape, Y_cg.shape, knn_xy.shape, knn_xx.shape)
print(knn_xx.sum(axis=1)) # weighted ka=5, p=0.7

(32324, 6244) (233514, 9364) (233514, 9364) (6244, 9364) (6244, 6244)
[[ 1.]
 [ 1.]
 [ 1.]
 ..., 
 [ 1.]
 [ 1.]
 [ 1.]]


# Compute metacell level signal 
- takes 270s (4.5 mins; run) + 145s (3 min; convert to non-sparse matrix) to run
- 88G of memory (does it make sense?)

In [22]:
ti = time.time()

# gene by metacell
gc_rna = X.dot(knn_xx.T).todense() 

# enhancer by metacell
ec_cg = Y_cg.dot(knn_xy.T).todense() 
ec_mcg = Y_mcg.dot(knn_xy.T).todense()  

print(time.time()-ti)

print(gc_rna.shape, ec_cg.shape, ec_mcg.shape)

108.1030125617981
(32324, 6244) (233514, 6244) (233514, 6244)


In [24]:
# # check sparsity level 
# def get_sparsiness(a):
#     m, n = a.shape
#     return a.nnz/(m*n)

# print(
#     get_sparsiness(X), 
#     get_sparsiness(Y_mcg), 
#     get_sparsiness(Y_cg), 
    
#     get_sparsiness(gc_rna), 
#     get_sparsiness(ec_cg), 
#     get_sparsiness(ec_mcg), 
#     )

In [25]:
# get mcc
ti = time.time()
ec_mccg =  snmcseq_utils.get_mcc_lite_v4(
                                pd.DataFrame(ec_cg).astype(np.float32), 
                                pd.DataFrame(ec_mcg).astype(np.float32), 
                                base_call_cutoff=5, sufficient_coverage_fraction=0.8, fillna=True)
print(ec_mccg.shape)

print(time.time()-ti)

(87943, 6244)
423.76569294929504


In [26]:
# save ec_mccg and gc_rna
ti = time.time()
f = '/cndd2/fangming/projects/scf_enhancers/results/{}_ec_mccg_test1.pkl'.format(today)
with open(f, 'wb') as fh:
    pickle.dump(ec_mccg, fh)
print(time.time()-ti)
    
ti = time.time()
f = '/cndd2/fangming/projects/scf_enhancers/results/{}_gc_rna_test1.pkl'.format(today)
with open(f, 'wb') as fh:
    pickle.dump(gc_rna, fh)
print(time.time()-ti)

5.454484224319458
3.887183904647827


# correlate enhancer and gene (a new notebook)

In [28]:
# load from file

# save ec_mccg and gc_rna
ti = time.time()
f = '/cndd2/fangming/projects/scf_enhancers/results/{}_ec_mccg_test1.pkl'.format(today)
with open(f, 'rb') as fh:
    ec_mccg = pickle.load(fh)
print(time.time()-ti)
    
ti = time.time()
f = '/cndd2/fangming/projects/scf_enhancers/results/{}_gc_rna_test1.pkl'.format(today)
with open(f, 'rb') as fh:
    gc_rna = pickle.load(fh)
print(time.time()-ti)

print(ec_mccg.shape, gc_rna.shape)

1.3835349082946777
1.0300688743591309
(87943, 6244) (32324, 6244)


In [29]:
# gene-cell matrix (rna)

ti = time.time()
f = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/smarter_cells_features.npz'
fgene = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/smarter_cells_features.gene'
fcell = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/smarter_cells_features.cell'

gene_cell_rna_mat = snmcseq_utils.load_gc_matrix(fgene, fcell, f)

print(gene_cell_rna_mat.data.shape)
print(time.time()-ti)


(32324, 6244)
9.320297718048096


In [30]:
# enhancer-gene linkage
ti = time.time()
f = '/cndd2/fangming/projects/scf_enhancers/results/200521_to_evals.tsv'
enhancer_gene_to_eval = pd.read_csv(f, sep='\t')
f = '/cndd2/fangming/projects/scf_enhancers/results/200521_to_evals_shuffled.tsv'
enhancer_gene_to_eval_shuffled = pd.read_csv(f, sep='\t')

print(enhancer_gene_to_eval.shape)
print(time.time()-ti)
enhancer_gene_to_eval.head()

(2589994, 18)
8.236989259719849


gene     ens       val      val2  val_clustershuffled  \
0  ENSMUSG00000000001  147520 -0.157895  0.127913            -0.169712   
1  ENSMUSG00000000001  147522  0.027465 -0.307145             0.064887   
2  ENSMUSG00000000001  147523 -0.040376  0.011489            -0.155487   
3  ENSMUSG00000000001  147527 -0.070795  0.111719            -0.088084   
4  ENSMUSG00000000001  147528  0.012583  0.150673            -0.036218   

   val2_clustershuffled    dist   pvalues  fdr<100kb  fdr<200kb  fdr<500kb  \
0             -0.049349  995118  0.288079   0.674683   0.736811   0.827139   
1              0.105591  985740  0.561881   0.830849   0.867082   0.917381   
2              0.014772  983725  0.457686   0.780196   0.825259   0.889141   
3              0.184594  970444  0.411564   0.755088   0.804341   0.874375   
4              0.147609  969372  0.539123   0.819981   0.858026   0.911467   

   fdr<100kb shuffled (cluster label)  pvalues2  fdr2<100kb  fdr2<200kb  \
0                            1.647931  0.383141    0.745977    0.797076   
1                            0.958300  0.935264    0.973809    0.979844   
2                            1.089312  0.580703    0.841673    0.874844   
3                            1.183332  0.410358    0.761056    0.809464   
4                            0.979843  0.345993    0.725803    0.780447   

   fdr2<500kb  fdr2<100kb shuffled (cluster label)  val_both  
0    0.872822                             1.955898 -0.285808  
1    0.987476                             0.959470  0.334610  
2    0.922832                             1.273339 -0.051866  
3    0.880982                             1.812516 -0.182514  
4    0.862033                             2.191506 -0.138090

In [31]:
# use ec_mccg and gc_rna
# correlate e-g according to a e-g table
# do the same to a shuffled e-g table

gene_idx = snmcseq_utils.get_index_from_array(gene_cell_rna_mat.gene, enhancer_gene_to_eval['gene'])
enh_idx = snmcseq_utils.get_index_from_array(ec_mccg.index.values, enhancer_gene_to_eval['ens']) # be careful here!

to_correlate = ~np.logical_or(gene_idx==-1, enh_idx==-1)
gene_idx = gene_idx[to_correlate]
enh_idx = enh_idx[to_correlate]
print(gene_idx.shape, enh_idx.shape)

(1543535,) (1543535,)


In [ ]:
# running time (45 mins)

# uses large amount of memory for some reason
ti = time.time()
corrs = snmcseq_utils.pearsonr_paired_rows_fast(gc_rna[gene_idx], ec_mccg.values[enh_idx])
print(time.time()-ti)

In [ ]:
gene_idx = snmcseq_utils.get_index_from_array(gene_cell_rna_mat.gene, enhancer_gene_to_eval_shuffled['gene'])
enh_idx = snmcseq_utils.get_index_from_array(ec_mccg.index.values, enhancer_gene_to_eval_shuffled['ens']) # be careful here!

to_correlate = ~np.logical_or(gene_idx==-1, enh_idx==-1)
gene_idx = gene_idx[to_correlate]
enh_idx = enh_idx[to_correlate]
print(gene_idx.shape, enh_idx.shape)

In [ ]:
# running time (45 mins)

# uses large amount of memory for some reason
ti = time.time()
corrs_shuffled = snmcseq_utils.pearsonr_paired_rows_fast(gc_rna[gene_idx], ec_mccg.values[enh_idx])
print(time.time()-ti)

# plotting results

In [ ]:
config = {
    'kde': False,
    "hist_kws": {
                'histtype': 'step', 
#                 'edgecolor': 'none',
                'alpha': 1, 
                'density': True, 
                },
}


fig, ax = plt.subplots()
num_bins = 200
bins = np.linspace(-0.2, 0.2, num_bins)
sns.distplot(corrs, 
             bins=bins, ax=ax, label='linked pairs (<500kb)', **config)
sns.distplot(corrs_shuffled, 
             bins=bins, ax=ax, label='shuffled pairs', **config)
ax.legend(bbox_to_anchor=(1,1))
plt.show()